## Multi-label Classification

### Imports

In [1]:
import pandas as pd
import tensorflow as tf

### Training (on raw dataset)

In [2]:
train = pd.read_csv("data_processed.csv")

features = ['age', 'admissionheight', 'admissionweight', 'hospitalid', 'patientunitstayid', 'intubated', 'vent', 'dialysis', 
'eyes', 'motor', 'verbal', 'apachepatientresultsid', 'acutephysiologyscore', 'apachescore', 'actualiculos', 'actualhospitallos', 
'ptcawithin24h', 'unabridgedunitlos', 'unabridgedhosplos', 'gender_Female', 'gender_Male', 'ethnicity_African American', 
'ethnicity_Asian', 'ethnicity_Caucasian', 'ethnicity_Hispanic', 'ethnicity_Native American', 'ethnicity_Other/Unknown', 
'actualhospitalmortality_ALIVE', 'actualhospitalmortality_EXPIRED']
targets = ['heart_problems', 'lung_problem', 'lung_infection', 'kidney_problems']

x_train = train.loc[:, features].to_numpy()
y_train = train.loc[:, targets].to_numpy()


model = tf.keras.models.Sequential([tf.keras.layers.Dense(4, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(4, activation=tf.nn.sigmoid)])

m = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(), 
tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=m)
history_1 = model.fit(x_train, y_train, validation_split=0.3, epochs=10, verbose=0)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-12-04 11:34:11.396264: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-04 11:34:11.396416: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-12-04 11:34:11.618281: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-04 11:34:12.529504: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-04 11:34:13.985982: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


### Training (on processed dataset)

In [3]:
df1 = pd.read_csv("data_processed.csv")
df2 = pd.read_csv("engineered_features.csv")
dataset = df1.merge(df2, right_on="patientunitstayid", left_on="patientunitstayid")

features = ['age', 'admissionheight', 'admissionweight', 'hospitalid',
       'patientunitstayid', 'intubated', 'vent', 'dialysis', 'eyes', 'motor',
       'verbal', 'apachepatientresultsid', 'acutephysiologyscore',
       'apachescore', 'actualiculos', 'actualhospitallos', 'ptcawithin24h',
       'unabridgedunitlos', 'unabridgedhosplos',  'gender_Female',
       'gender_Male', 'ethnicity_African American', 'ethnicity_Asian',
       'ethnicity_Caucasian', 'ethnicity_Hispanic',
       'ethnicity_Native American', 'ethnicity_Other/Unknown',
       'actualhospitalmortality_ALIVE', 'actualhospitalmortality_EXPIRED',
       'creatinine_comparison', 'bun_comparison', 'bun_creatinine_comparison',
       'calcium_comparison', 'hco3_comparison', 'phosphate_comparison',
       'glucose_comparison', 'potassium_comparison', 'magnesium_comparison',
       'sodium_comparison', 'chloride_comparison', 'paCO2_comparison',
       'paO2_comparison']
targets = ['heart_problems', 'lung_problem', 'lung_infection', 'kidney_problems']

x_train = dataset.loc[:, features].to_numpy()
y_train = dataset.loc[:, targets].to_numpy()


model = tf.keras.models.Sequential([tf.keras.layers.Dense(4, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(4, activation=tf.nn.sigmoid)])

m = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(), 
tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=m)
history_2 = model.fit(x_train, y_train, validation_split=0.3, epochs=10, verbose=0)

2022-12-04 11:34:22.601759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-04 11:34:23.955811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


### Scores (accuracy, precision, recall, sensitivity, specitivity)

In [6]:
his_1 = pd.DataFrame(history_1.history)
his_2 = pd.DataFrame(history_2.history)

In [12]:
def get_sens(tp, fn):
    return [tp/(tp+fn)]
def get_spec(tn, fp):
    return [tn/(tn+fp)]

normal_ds_train = pd.DataFrame({"accuracy":[0.969722],"precision":[0.729184],"recall":[0.665388],"sensitivity":get_sens(867.0,436.0),
                    "specificity":get_spec(3131.0,322.0)})
normal_ds_val = pd.DataFrame({"accuracy":[0.954902],"precision":[0.739216],"recall":[0.612013],"sensitivity":get_sens(377.0,239.0),
                    "specificity":get_spec(1291.0,133.0),})

engineered_ds_train = pd.DataFrame({"accuracy":[0.969388],"precision":[0.727891],"recall":[0.663566],"sensitivity":get_sens(856.0,434.0),
                    "specificity":get_spec(3094.0,320.0),})
engineered_ds_val = pd.DataFrame({"accuracy":[0.956436],"precision":[0.740594],"recall":[0.612111],"sensitivity":get_sens(374.0,237.0),
                    "specificity":get_spec(1278.0,131.0),})

In [25]:
info = pd.DataFrame()
info = pd.concat([normal_ds_train, normal_ds_val, engineered_ds_train, engineered_ds_val], axis=0)
info["name"] = ["Normal (train)", "Normal (test)", "Engineered (train)", "Engineered (test)"]
info.insert(0, 'name', info.pop('name'))
# info.to_csv("scores.csv", index=False)
info.style.hide(axis='index')

name,accuracy,precision,recall,sensitivity,specificity
Normal (train),0.969722,0.729184,0.665388,0.665388,0.906748
Normal (test),0.954902,0.739216,0.612013,0.612013,0.906601
Engineered (train),0.969388,0.727891,0.663566,0.663566,0.906268
Engineered (test),0.956436,0.740594,0.612111,0.612111,0.907026
